### María Sofía Álvarez - Brenda Barahona - Álvaro Plata
<h1 align='center'>Proyecto 1: Analítica de textos - Preprocesamiento</h1>

En esta fase del proyecto, nos encargaremos de construir las pipelines necesarias para desplegar nuestros modelos en una API. Para ello, únicamente nos remitiremos a construir los 3 modelos que nos dieron los mejores resultados en la fase 1 de este proyecto, después de realizar el ajuste de hiperparámetros desarrollado en la fase anterior. Este notebook se divide en ciertas fases. Primero, se realiza todo el preprocesamiento, que es común a todos los modelos. Segundo, se terminan de construir las pipelines propias de cada modelo, de acuerdo con los hiperparámetros que fueron ajustados en la fase 1 (solo con ellos, para ahorrar tiempo de cómputo). Por último, se exportan los tres modelos.

Si desea ver a fondo alguna parte del perfilamiento, preprocesamiento de datos realizado, o ajuste de hiperparámetrod, remítase al repositorio de github de la fase 1 de este proyecto, disponible en <a href="https://github.com/sofiaalvarezlopez/Proyecto-1-BI">este link</a>.

## Importación de librerías
Importamos las librerías necesarias para el desarrollo de este proyecto. Las librerías instaladas son exactamente las mismas que las que se instalaron para la fase 1 de este proyecto.

In [26]:
# ESAI
import re
import nltk
import keras
import spacy
import inflect
import sent2vec # Para descargar esta libreria, es necesario descargarla desde GitHub https://github.com/epfml/sent2vec
import stopwords
import numpy as np
import unicodedata
import pandas as pd
import contractions
import seaborn as sns
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import pandas_profiling as pp
from joblib import dump, load
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import ComplementNB
from nltk import word_tokenize, sent_tokenize
from tensorflow.keras.models import Sequential
from sklearn.utils import resample, class_weight
from keras.wrappers.scikit_learn import KerasClassifier
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.metrics import precision_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from clases import Preprocessing, StemAndLemmatize, VectorizeLSTM, LSTMBuilder
from keras.layers import LSTM, Dense, Embedding, TextVectorization, Input, Dropout

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [4]:
%matplotlib inline

Procedemos, entonces, a ver los datos suministrados. 

In [5]:
diagnoses =pd.read_csv('ApoyoDiagnosticoEstudiante/medical_text_clasificacion.csv')
X, Y = diagnoses.drop(['problems_described'], axis=1), diagnoses['problems_described']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,stratify=Y,test_size=0.95, random_state=28)
data_train = pd.concat( [X_train, Y_train], axis=1)

## Preprocesamiento
Iniciamos con la fase de preprocesamiento de los datos:
### Extracción de entidades médicas
De acuerdo con [4], para clasificación en contextos médicos es útil extraer entidades médicas para la clasificación, como se hizo en el proyecto pasado. No obstante, se obtuvieron mejores resultados con las entidades en todos los casos, así que se omitirá este pedazo:
```python
nlp = en_ner_bionlp13cg_md.load()
def medical_entities(text):
    entities = []
    doc = nlp(text)
    for ent in doc.ents:
        entities.append(ent.text)
    return ' '.join(entities)
```


### Manejo de Ruido 
En esta sección se quitará o modificará todo lo que se considere como ruido:

+ Caracteres no ascii: Hace parte importante del preprocesamiento de las palabras. Con caracteres no-ascii, el preprocesamiento puede verse terriblemente perjudicado.
+ Se pasará de mayusculas a minusculas: Asimismo, es importante que todas las palabras tengan una capitalización homogénea (en este caso, queremos que estén en minúscula).
+ Se eliminará la puntuación: Por otro lado, consideramos que la puntuación no provee información adicional en este contexto. Adicionalmente, de no eliminarse, puede aumentar la dimensionalidad de los datos sin proveer más información. Por ejemplo, no tiene sentido pensar que "almuerzo!" y "almuerzo" sean palabras diferentes. Por ello removemos toda la puntuación usando expresiones regulares.
+ Se reemplazarán los números: Ahora, podemos suponer que los números no proveen información relevante para el problema en cuestión. Estos pueden también agregar dimensionalidad inutilmente al problema.
+ Se quitarán las fechas (si las hay) también: las fechas son irrelevantes para el contexto del problema.
+ Se quitarán las palabras vacias (artículos, pronombres, preposiciones): Estas se denominan stop-words, en inglés. Son palabras que se usan en muchos contextos (como 'the') y no aportan información significativa en la construcción del modelo. Asimismo, definimos nuestras propias stopwords de acuerdo con el perfilamiento realizado, pues son palabras que no aportan significativamente al contexto.

```python
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words
    

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_numbers(words):
    p = inflect.engine()
    new_words = []
    for word in words:
        new_word = re.sub('\d+.*', '', word)
        if not word.isnumeric() and new_word != '':
            new_words.append(word)
    return new_words

def remove_dates(words):
    """Replace all dates in our data"""
    new_words = []
    for word in words:
        new_word = re.sub(r'\d+/\d+/\d+', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words
```


De todas estas palabras, encontramos que solamente tumor y lesion pueden ser relevantes para nuestro análisis. Por lo tanto, las quitamos todas, excepto estas dos. Además, en un análisis preliminar encontramos otras palabras irrelevantes, las cuales también eliminamos.

```python
#En una primera iteracion nos dimos cuenta que las palabras "paty", "patients" aparece frecuentemente en todas las enfermedades,
#estos serán eliminados por que no agregan información valiosa. 
our_stopwords = ["paty","patients","p","study","result", "human", "humans", "monkey", "monkeys", 
                 "diseases", "studied","first", "rat", "patient", "case", "p less", "treatment", 
                 "group", "associated", "result", "may", "effect", "compared", "use", "cases", "year", 
                 "years", "age", "study", "disease", "found", "normal", "month", "although", "per cent",
                 "one", "two", "three", "four", "n", "children", "women"]

def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english') and word not in our_stopwords:
            new_words.append(word)
    return new_words
```
Asimismo, tenemos la función de eliminación del ruido global:

``` python
def noise_elimination(words):
    words = to_lowercase(words)
    words = remove_non_ascii(words)
    words = remove_numbers(words)
    words = remove_dates(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words
```




Con esto, ya tenemos casi listo nuestro proceso de eliminación del ruido. Primero, llamamos a la función <code>fix</code> de la librería ```contraction``` para aquellas contracciones que no están separadas en dos palabras. Esta elimina todas las ocurrencias de contracciones en inglés, reemplazándolas por su equivalente sin contracción. Una vez realizado este paso, "tokenizamos" las historias clínicas. Para poder evaluar cada palabra por separado y aplicar los pasos de preprocesamiento, hacemos la tokenización en palabras individuales usando el módulo ```word_tokenize```. Finalmente, aplicamos la función```noise_elimination``` definida previamente.

Más adelante, lo que realmente nos servirá será volver a tener los documentos sin tokenización para el proceso de vectorización (sea tf-idf, o BioSentVec, como se vera mas adelante). Entonces, volvemos a juntar todas las palabras para cada documento y retornamos eso. También retornamos las palabras tokenizadas con el fin de realizar la lematización estemización más adelante.

Note que estas funciones las aplicamos tanto sobre los medical abstracts iniciales, como las palabras clave que obtuvimos con la librería de SpaCy descrita previamente.

```python
def preprocessing(X):
    new_X_train= X.apply(contractions.fix) #Aplica la corrección de las contracciones
    new_X_train = new_X_train.apply(word_tokenize)
    new_X_train = new_X_train.apply(noise_elimination) #Aplica la eliminación del ruido
    X_train = new_X_train.apply(lambda x: ' '.join(map(str, x)))
    return new_X_train, X_train
```

### Normalización: Stemming y Lemmatization 
Aplicaremos tecnicas como "Stemming" y "Lemmatization" sobre la columna "medical_abstracts" y "medical_entities". 

En esta parte del preprocesamiento, hacemos una eliminación de prefijos y sufijos, así como una lematización de los verbos. En el caso del Stemming hay varios algoritmos que podemos utilizar: Porter, Snowball (Porter2) o Lancaster (Paice-Husk). De acuerdo a lo que encontramos,  <a href="https://stackoverflow.com/questions/10554052/what-are-the-major-differences-and-benefits-of-porter-and-lancaster-stemming-alg"> la agresividad en el corte de raíces de las palabras de estos algoritmos aumenta, siendo Porter el menos agresivo y Lancaster el más agresivo </a>. En este sentido, parece ser que Lancaster (a pesar de ser el más eficiente de todos), puede ser poco riguroso y así crear muchas ambigüedades. Asimismo, Porter2 es un poco más agresivo que Porter, sin perder mucho el origen de las palabras y con un tiempo de cómputo razonable. El mismo Porter, creador del algoritmo, argumenta que es una mejora de su algoritmo original. Con el fin de tener la mejor preparación de las palabras, en un tiempo de cómputo razonable, decidimos usar Porter2. En el caso de la lematización, sí usamos WordNetLemmatizer() al ser el más usado en el mundo del procesamiento de textos.
```python
#Funciones de "Stemming" y "Lemmatization"
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = SnowballStemmer('english')
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas
```

Y creamos la clase correspondiente para las funciones:
``` python
class StemAndLemmatize():
    def __init__(self):
        pass
    def transform(self,X,y=None):
        stems_abs = stem_words(X["medical_abstracts"])
        lemmas_abs = lemmatize_verbs(X["medical_abstracts"])
        stems_ents = stem_words(X["medical_entities"])
        lemmas_ents = lemmatize_verbs(X["medical_entities"])
        return X
    def fit(self, X, y=None):
        return self
```

Por último, es posible que, tras el preprocesamiento, algunos datos hayan quedado con entidades médicas nulas. Entonces, es necesario eliminar estas filas. 

Finalmente, con todas las funciones definidas, creamos el pipeline de preprocesamiento:

In [6]:
preproc = [
    ("preprocessing", Preprocessing()),
    ("stem_lemmatize", StemAndLemmatize())
]

In [7]:
pipe  = Pipeline(preproc)

In [8]:
datos_train_procesados = pipe.fit_transform(data_train, data_train['problems_described'])
datos_train_procesados.head(5)

,medical_abstracts,problems_described
5157,"[biliari, gut, function, follow, shock, aim, c...",5
4803,"[inpati, theophyllin, toxic, prevent, factor, ...",5
9804,"[transor, approach, manag, intradur, lesion, c...",3
7183,"[control, trial, communiti, base, coronari, re...",4
9719,"[high, preval, antibodi, hepat, c, virus, hepa...",1


Veamos una muestra de los textos preprocesados:

Finalmente, almacenamos los datos de entrenamiento en un archivo <code>.csv</code> denominado: <code>proyecto1_fase2_datos_entrenamiento.csv</code>:

In [9]:
datos_train_procesados.to_csv("proyecto1_fase2_datos_preprocesados.csv")

## Modelo: LSTM
En la iteración pasada, diseñamos tres algoritmos de Machine Learning para probar: Naïve-Bayes, OneVsRest y una red neuronal usando una LSTM (Long-Short Term Memory). De estos, el que mejores métricas arrojó fue el LSTM. Por lo tanto, debido a que buscamos optimizar la precisión de nuestro modelo para el cliente, este es el que desplegaremos en la API. Si desea ver más información acerca de la LSTM, remítase al repositorio del proyecto anterior.

In [10]:
diagnoses =pd.read_csv('ApoyoDiagnosticoEstudiante/medical_text_clasificacion.csv')
X, Y = diagnoses.drop(['problems_described'], axis=1), diagnoses['problems_described']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,stratify=Y,test_size=0.95, random_state=28)
data_train = pd.concat( [X_train, Y_train], axis=1)

### Manejo de desbalanceo de las clases

Ahora, uno de los mayores problemas de la clasificación es el contexto desbalanceado. Una opción sería reducir el conjunto de datos hasta que todas las clases queden con un número de abstracts igual al tamaño de la clase de menor cantidad de abstracts. No obstante, por lo general la idea es no reducir el conjunto de datos. Otra opción, como en los algoritmos anteriores, sería usar SMOTE. No obstante, esto es computacionalmente muy costoso para la red.

Lo que sí podemos hacer es considerar pesos. Así, el modelo podrá prestar mayor atención a las clases minoritarias. Para ello, usaremos la librería de <code>sk-learn</code> y lo pasaremos como un objeto al modelo que construiremos más adelante:

In [11]:
class_weights = class_weight.compute_class_weight(
                class_weight = 'balanced',
                classes = np.unique(diagnoses['problems_described']), 
                y = diagnoses['problems_described'])
train_class_weights_ = dict(enumerate(class_weights, start=1))
train_class_weights = dict(enumerate(class_weights))

Podemos ver los pesos asociados a cada una de las clases, en orden ascendente:

In [12]:
train_class_weights_

{1: 0.9128946367440092,
 2: 1.932367149758454,
 3: 1.5,
 4: 0.9463722397476341,
 5: 0.6010518407212622}

### Modelo

Usando el modelo de vectorización de BioSentVec, se obtuvo, utilizando la búsqueda de hiperparámetros, que el mejor modelo era aquel que no tenía ninguna capa adicional a las consideradas inicialmente (dos capas LSTM con sus respectivas capas de dropout), un dropout de 0.1 (i.e. una tasa de pérdida bastante pequeña) y cada una de las capas con 64 neuronas.

Finalmente, se tiene una capa softmax con 5 neuronas, que corresponden con las 5 clases del problema. Es importante mencionar que la variable categórica Y se debe convertir a la funcionalidad de Keras para el correcto funcionamiento de la red.

In [13]:
X, Y = data_train.drop(['problems_described'], axis=1), data_train['problems_described']
Y = keras.utils.np_utils.to_categorical(Y)[:,1:]

Y, por último, creamos la vectorización:

In [14]:
pipe.steps.append(('vectorizer', VectorizeLSTM()))

Veamos qué tiene la pipeline hasta ahora:

In [15]:
pipe

Pipeline(steps=[('preprocessing',
                 <clases.Preprocessing object at 0x7f7e68e48670>),
                ('stem_lemmatize',
                 <clases.StemAndLemmatize object at 0x7f7e68e481f0>),
                ('vectorizer',
                 <clases.VectorizeLSTM object at 0x7f7e68e4ac40>)])

Finalmente, creamos un clasificador de Keras usando el modelo:
```python
class LSTMBuilder():
    def __call__(self):
        output=5
        model = Sequential(name="LSTM")
        # Agregamos una capa LSTM con el tamanio de entrada de los embedded abstracts y 64 neuronas en la capa
        model.add(LSTM(units=64, return_sequences=True, 
                    input_shape=(1, 700)))
        model.add(Dropout(0.1))
        # Agregamos una segunda capa LSTM con 16 neuronas
        model.add(LSTM(units=64, return_sequences=False))
        # Con su respectiva capa de dropout
        model.add(Dropout(0.1))
        # Definimos la capa de salida
        model.add(Dense(output, activation='softmax'))
        # Compilo
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.Precision(name='precision')])
        return model
```

In [16]:
clf = KerasClassifier(LSTMBuilder(), verbose=1)

In [17]:
pipe.steps.append((('model', clf)))

In [19]:
pipe_elegida = pipe.fit(X, Y, 
                        model__class_weight=train_class_weights,
                        model__validation_split=0.2, 
                        model__epochs= 100)

Model successfuly loaded


2022-05-10 23:22:53.851758: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
15/15 [==============================] - 3s 58ms/step - loss: 1.6324 - precision: 0.0000e+00 - val_loss: 1.6063 - val_precision: 0.0000e+00
Epoch 2/100
15/15 [==============================] - 0s 4ms/step - loss: 1.5974 - precision: 0.0000e+00 - val_loss: 1.5732 - val_precision: 0.0000e+00
Epoch 3/100
15/15 [==============================] - 0s 4ms/step - loss: 1.4874 - precision: 0.0000e+00 - val_loss: 1.4706 - val_precision: 0.0000e+00
Epoch 4/100
15/15 [==============================] - 0s 4ms/step - loss: 1.2607 - precision: 0.7286 - val_loss: 1.2905 - val_precision: 0.6667
Epoch 5/100
15/15 [==============================] - 0s 4ms/step - loss: 1.0310 - precision: 0.7130 - val_loss: 1.2113 - val_precision: 0.6562
Epoch 6/100
15/15 [==============================] - 0s 4ms/step - loss: 0.8752 - precision: 0.6962 - val_loss: 1.1051 - val_precision: 0.6364
Epoch 7/100
15/15 [==============================] - 0s 4ms/step - loss: 0.7627 - precision: 0.7364 - val_loss: 1.079

Epoch 58/100
15/15 [==============================] - 0s 4ms/step - loss: 0.0265 - precision: 0.9916 - val_loss: 2.0386 - val_precision: 0.5690
Epoch 59/100
15/15 [==============================] - 0s 3ms/step - loss: 0.0359 - precision: 0.9916 - val_loss: 2.0005 - val_precision: 0.5913
Epoch 60/100
15/15 [==============================] - 0s 4ms/step - loss: 0.0294 - precision: 0.9896 - val_loss: 2.1105 - val_precision: 0.5702
Epoch 61/100
15/15 [==============================] - 0s 3ms/step - loss: 0.0313 - precision: 0.9895 - val_loss: 2.1287 - val_precision: 0.5664
Epoch 62/100
15/15 [==============================] - 0s 3ms/step - loss: 0.0281 - precision: 0.9875 - val_loss: 2.1308 - val_precision: 0.5690
Epoch 63/100
15/15 [==============================] - 0s 3ms/step - loss: 0.0325 - precision: 0.9875 - val_loss: 2.1025 - val_precision: 0.5812
Epoch 64/100
15/15 [==============================] - 0s 3ms/step - loss: 0.0374 - precision: 0.9854 - val_loss: 2.0908 - val_precision:

### Prueba de la calidad del modelo
Veamos la matriz de confusión y otras métricas pertinentes para este modelo. Para un análisis más exhaustivo, remítase al anterior proyecto.

### Obtención de las estadísticas
Veamos la probabilidad de que un texto determinado pertenezca a cada clase:

In [21]:
pred = pipe_elegida.predict_proba(X)

Model successfuly loaded


### Exportación del modelo
Una vez entrenado el modelo, procedemos a guardarlo y exportarlo. Debido a que no existe una manera directa de almacenar un modelo de keras dentro de una pipeline de scikit learn, es necesario guardarlos por separado. Así, guardamos primero el modelo de Keras:

In [34]:
pipe_elegida.named_steps['model'].model.save('./assets/keras_model.h5')
pipe_elegida.named_steps['model'].model = None
dump(pipe_elegida, './assets/modelo.pkl')

AttributeError: 'NoneType' object has no attribute 'save'

In [35]:
pred[0]

array([5.0689712e-05, 4.0981346e-03, 1.7193420e-03, 2.3285644e-04,
       9.9389899e-01], dtype=float32)

In [42]:
X

,medical_abstracts
5157,biliari gut function follow shock aim charact ...
4803,inpati theophyllin toxic prevent factor object...
9804,transor approach manag intradur lesion craniov...
7183,control trial communiti base coronari rehabili...
9719,high preval antibodi hepat c virus hepatocellu...
...,...
5017,essenti hyperten sign search concept cardin im...
9129,acut toler morphin analgesia continu infus sin...
6539,hepat lesion rabbit induc acoust cavit tissu d...
714,flumazenil neonat side benzodiazepin pregnanc ...


<h2 id='bibliografia'>Bibliografía</h2>

---

<a id='geron'>[1]</a> Géron, A. (2017). Hands-on machine learning with Scikit-Learn and TensorFlow : concepts, tools, and techniques to build intelligent systems. Sebastopol, CA: O'Reilly Media. ISBN: 978-1491962299

<a id='nlp_profiler'>[2]</a> https://towardsdatascience.com/nlp-profiler-profiling-datasets-with-one-or-more-text-columns-9b791193db89

[3] https://www.kaggle.com/code/neomatrix369/nlp-profiler-simple-dataset/notebook

[4] Clinical Text Classification: https://www.kaggle.com/ritheshsreenivasan/clinical-text-classification